# Starbucks Capstone Challenge

### Introduction

This data set contains simulated data that mimics customer behavior on the Starbucks rewards mobile app. Once every few days, Starbucks sends out an offer to users of the mobile app. An offer can be merely an advertisement for a drink or an actual offer such as a discount or BOGO (buy one get one free). Some users might not receive any offer during certain weeks. 

Not all users receive the same offer, and that is the challenge to solve with this data set.

Your task is to combine transaction, demographic and offer data to determine which demographic groups respond best to which offer type. This data set is a simplified version of the real Starbucks app because the underlying simulator only has one product whereas Starbucks actually sells dozens of products.

Every offer has a validity period before the offer expires. As an example, a BOGO offer might be valid for only 5 days. You'll see in the data set that informational offers have a validity period even though these ads are merely providing information about a product; for example, if an informational offer has 7 days of validity, you can assume the customer is feeling the influence of the offer for 7 days after receiving the advertisement.

You'll be given transactional data showing user purchases made on the app including the timestamp of purchase and the amount of money spent on a purchase. This transactional data also has a record for each offer that a user receives as well as a record for when a user actually views the offer. There are also records for when a user completes an offer. 

Keep in mind as well that someone using the app might make a purchase through the app without having received an offer or seen an offer.

### Example

To give an example, a user could receive a discount offer buy 10 dollars get 2 off on Monday. The offer is valid for 10 days from receipt. If the customer accumulates at least 10 dollars in purchases during the validity period, the customer completes the offer.

However, there are a few things to watch out for in this data set. Customers do not opt into the offers that they receive; in other words, a user can receive an offer, never actually view the offer, and still complete the offer. For example, a user might receive the "buy 10 dollars get 2 dollars off offer", but the user never opens the offer during the 10 day validity period. The customer spends 15 dollars during those ten days. There will be an offer completion record in the data set; however, the customer was not influenced by the offer because the customer never viewed the offer.

### Cleaning

This makes data cleaning especially important and tricky.

You'll also want to take into account that some demographic groups will make purchases even if they don't receive an offer. From a business perspective, if a customer is going to make a 10 dollar purchase without an offer anyway, you wouldn't want to send a buy 10 dollars get 2 dollars off offer. You'll want to try to assess what a certain demographic group will buy when not receiving any offers.

### Final Advice

Because this is a capstone project, you are free to analyze the data any way you see fit. For example, you could build a machine learning model that predicts how much someone will spend based on demographics and offer type. Or you could build a model that predicts whether or not someone will respond to an offer. Or, you don't need to build a machine learning model at all. You could develop a set of heuristics that determine what offer you should send to each customer (i.e., 75 percent of women customers who were 35 years old responded to offer A vs 40 percent from the same demographic to offer B, so send offer A).

# Data Sets

The data is contained in three files:

* portfolio.json - containing offer ids and meta data about each offer (duration, type, etc.)
* profile.json - demographic data for each customer
* transcript.json - records for transactions, offers received, offers viewed, and offers completed

Here is the schema and explanation of each variable in the files:

**portfolio.json**
* id (string) - offer id
* offer_type (string) - type of offer ie BOGO, discount, informational
* difficulty (int) - minimum required spend to complete an offer
* reward (int) - reward given for completing an offer
* duration (int) - time for offer to be open, in days
* channels (list of strings)

**profile.json**
* age (int) - age of the customer 
* became_member_on (int) - date when customer created an app account
* gender (str) - gender of the customer (note some entries contain 'O' for other rather than M or F)
* id (str) - customer id
* income (float) - customer's income

**transcript.json**
* event (str) - record description (ie transaction, offer received, offer viewed, etc.)
* person (str) - customer id
* time (int) - time in hours since start of test. The data begins at time t=0
* value - (dict of strings) - either an offer id or transaction amount depending on the record

**Note:** If you are using the workspace, you will need to go to the terminal and run the command `conda update pandas` before reading in the files. This is because the version of pandas in the workspace cannot read in the transcript.json file correctly, but the newest version of pandas can. You can access the termnal from the orange icon in the top left of this notebook.  

You can see how to access the terminal and how the install works using the two images below.  First you need to access the terminal:

<img src="pic1.png"/>

Then you will want to run the above command:

<img src="pic2.png"/>

Finally, when you enter back into the notebook (use the jupyter icon again), you should be able to run the below cell without any errors.

In [1]:
import pandas as pd
import numpy as np
import math
import json
import matplotlib.pyplot as plt
% matplotlib inline

# read in the json files
portfolio = pd.read_json('data/portfolio.json', orient='records', lines=True)
profile = pd.read_json('data/profile.json', orient='records', lines=True)
transcript = pd.read_json('data/transcript.json', orient='records', lines=True)

# breaking down value
values = transcript['value'].apply(pd.Series)
transcript = pd.concat([transcript, values], axis=1)
transcript.drop('value', axis = 1)

# concat offer_id with offer id
transcript['offer_id'] = transcript['offer_id'].replace(np.nan, '', regex=True)
transcript['offer id'] = transcript['offer id'].replace(np.nan, '', regex=True)
transcript['offer_id'] = transcript['offer_id'] + transcript['offer id']
transcript.drop('offer id', axis = 1, inplace = True)

# merge everything
transcript_profile = pd.merge(transcript, profile, how='left', left_on='person', right_on='id').drop('id', axis = 1)
df = pd.merge(transcript_profile, portfolio, how='left', left_on='offer_id', right_on='id').drop('id', axis = 1)
df.drop(['reward_x', 'value'], axis = 1, inplace = True)

# breaking down channels
channels = df['channels'].apply(pd.Series).rename(columns={0: "channels_1", 1: "channels_2", 2: "channels_3", 3: "channels_4"})
df = pd.concat([df, channels], axis=1).drop('channels', axis = 1)
for column in ['channels_1', 'channels_2', 'channels_3', 'channels_4']:
    df[column] = df.apply(lambda x: 0 if x[column] == np.nan else 1, axis = 1)
df.head()

,event,person,time,amount,offer_id,age,became_member_on,gender,income,difficulty,duration,offer_type,reward_y,channels_1,channels_2,channels_3,channels_4
0,offer received,78afa995795e4d85b5d9ceeca43f5fef,0,NaN,9b98b8c7a33c4b65b9aebfe6a799e6d9,75,20170509,F,100000.0,5.0,7.0,bogo,5.0,1,1,1,1
1,offer received,a03223e636434f42ac4c3df47e8bac43,0,NaN,0b1e1539f2cc45b7b9fa7c272da2e1d7,118,20170804,None,NaN,20.0,10.0,discount,5.0,1,1,1,1
2,offer received,e2127556f4f64592b11af22de27a7932,0,NaN,2906b810c7d4411798c6938adc9daaa5,68,20180426,M,70000.0,10.0,7.0,discount,2.0,1,1,1,1
3,offer received,8ec6ce2a7e7949b1bf142def7d0e0586,0,NaN,fafdcd668e3743c1bb461111dcafc2a4,118,20170925,None,NaN,10.0,10.0,discount,2.0,1,1,1,1
4,offer received,68617ca6246f4fbc85e91a2a49552598,0,NaN,4d5c57ea9a6940dd891ad53e9dbe8da0,118,20171002,None,NaN,10.0,5.0,bogo,10.0,1,1,1,1


In [3]:
df['amount'][0] == np.nan

False

In [4]:
df['amount'][0] == df['amount'][0]

False

# EDA

In [ ]:
profile['age'].value_counts(bins = 10).sort_index()

In [ ]:
profile[['age', 'income']].describe()

In [ ]:
profile.isnull().sum() / profile.shape[0]

In [ ]:
plt.hist(profile['gender'].dropna(), bins = 30)
plt.xlabel('Gender')
plt.show()

In [ ]:
plt.hist(profile['age'].dropna(), bins = 40)
plt.xlabel('Age')
plt.show()

In [ ]:
# # filtering technique
# age_lower = profile['age'] > 50
# age_upper = profile['age'] < 70
# profile[age_lower & age_upper].describe()

In [ ]:
transcript['event'].value_counts()

In [ ]:
plt.hist(transcript['time'], bins = 50)
plt.xlabel('time')
plt.show()

In [ ]:
df.isnull().sum()